<a href="https://colab.research.google.com/github/Juniormoraess/Pyomo---OPL/blob/master/Problema%20de%20fluxo%20de%20custo%20minimo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## Instalação do pyomo e do solver para o colab ##
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

In [0]:
## Caso a implementação esteja sendo feita por uma IDE: ##
## Abra o Anaconda Prompt e insira as seguintes chamadas: ##
## conda install -c conda-forge pyomo ##
## conda install -c conda-forge glpk ##

## Importando a biblioteca pyomo ##
import pyomo.environ as pyEnv

## Dados do problema ##
custos = [200, 900, 400, 300, 100, 300, 200]
fluxos = [50, 40, 0, -30, -60]
capacidades = [10, 0, 0, 0, 80, 0, 0]
arestas = [[1, 1, 1, 0, 0, 0, 0],
           [-1, 0, 0, 1, 0, 0, 0],
           [0, 0, -1, -1, 1, 0, 0],
           [0, -1, 0, 0, 0, 1, -1],
           [0, 0, 0, 0, -1, -1, 1]]

m = len(custos)
n = len(fluxos)

## Declarando o modelo através da biblioteca ##
modelo = pyEnv.ConcreteModel()

## Criando Indices ##
modelo.Indices = range(m)
modelo.Indices2 = range(n)

## Criando as variáveis ##
modelo.Variaveis = pyEnv.Var(modelo.Indices, within = pyEnv.NonNegativeReals)

## Criando a Função Objetivo ##
def Objetivo(modelo):
  return sum(custos[i]*modelo.Variaveis[i] for i in modelo.Indices)

modelo.Objetivo = pyEnv.Objective(rule = Objetivo, sense = pyEnv.minimize)

## Criando as restrições ##
def rest1(modelo, i):
  return sum(arestas[i][j]*modelo.Variaveis[j] for j in modelo.Indices) == fluxos[i]

def rest2(modelo, j):
      if (capacidades[j] > 0):
        return modelo.Variaveis[j] <= capacidades[j]
      else:
        return modelo.Variaveis[j] >= capacidades[j]
      
    
modelo.rest1 = pyEnv.Constraint(modelo.Indices2, rule = rest1)
modelo.rest2 = pyEnv.Constraint(modelo.Indices, rule = rest2)

## Forma manual da rest2 ##
##modelo.rest2 = pyEnv.Constraint(expr = (modelo.Variaveis[0] <= capacidades[0]))
##modelo.rest3 = pyEnv.Constraint(expr = (modelo.Variaveis[4] <= capacidades[4]))

## Chamando o Solver ##
solver = pyEnv.SolverFactory('glpk', executable = '/usr/bin/glpsol')
result_objetivo = solver.solve(modelo, tee = True)

## Printando o resultado ##
lista = list(modelo.Variaveis.keys())
print()
print()
print('Variaveis: ')
print()
for i in lista:
  print(i, '---', modelo.Variaveis[i]())
print()
print('Valor da função objetivo =', modelo.Objetivo())